In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langsmith.wrappers import wrap_openai
from langsmith import traceable
load_dotenv()

# read in env variables
api_key = os.getenv("GEMINI_API_KEY")
api_base = os.getenv("GEMINI_API_BASE")
model = os.getenv("GEMINI_API_MODEL")

In [2]:
os.getenv("LANGSMITH_PROJECT")

'week2-demo'

In [4]:
client = wrap_openai(OpenAI(
 api_key=api_key,
 base_url=api_base,
))

In [6]:
# single user message
messages=[
    {
    "role": "system",
    "content": "You are a helpful assistant who speaks like a pirate."
    },
    {
    "role": "user",
    "content": "A brief history of the internet?"
    }
]
# call the chat completion endpoint
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=150,
    temperature=0.5
)
print("=== Basic Chat Completion ===")
print(f"Response: {response.choices[0].message.content}")

=== Basic Chat Completion ===
Response: Ahoy there, matey! Ye be askin' about the grand ol' Internet, eh? Well, gather 'round, and I'll spin ye a yarn about its beginnings, as best a salty dog like meself can recall.

'Twas back in the days when the world was a bit more… disconnected, ye see. The seeds of this grand network were sown by the military, specifically the **United States Department of Defense**. They had this idea, back in the late 1960s, called **ARPANET**. The goal was to create a communication system that could still function even if some parts were knocked out, like in a war. Think of it as a way to keep the messages flow


In [7]:
@traceable(name="multi_turn_conversation")
def multi_turn_conversation():
    """Demonstrate multi-turn conversation with context preservation"""

    # Initialize conversation history with the system message
    conversation_history = [
        {
            "role": "system",
            "content": "You are a helpful programming tutor. Provide clear explanations and build upon previous topics in the conversation."
        }
    ]

    # now add the first user message to the history
    conversation_history.append(
        {
            "role": "user",
            "content": "What is a Python list?"
    })

    # now call the model to get the assistant's response
    response = client.chat.completions.create(
        model=model,
        messages=conversation_history,
        temperature=0.1
    )

    # add the response to the conversation history
    assistant_response = response.choices[0].message.content
    conversation_history.append({
        "role": "assistant",
        "content": assistant_response
    })

    # now ask a follow-up question
    conversation_history.append({
        "role": "user",
        "content": "How do I add items to it?"
    })

    response = client.chat.completions.create(
        model=model,
        messages=conversation_history,
        temperature=0.1
    )

    # now add the assistant's response to the history
    assistant_response = response.choices[0].message.content
    conversation_history.append({
        "role": "assistant",
        "content": assistant_response
    })

    # ask another follow-up question
    conversation_history.append({
        "role": "user",
        "content": "What's the difference between append() and extend()?"
    })

    response = client.chat.completions.create(
        model=model,
        messages=conversation_history,
        temperature=0.1
    )

    assistant_response = response.choices[0].message.content
    conversation_history.append({
        "role": "assistant",
        "content": assistant_response
    })

    return conversation_history



if __name__ == "__main__":
    print(multi_turn_conversation())

[{'role': 'system', 'content': 'You are a helpful programming tutor. Provide clear explanations and build upon previous topics in the conversation.'}, {'role': 'user', 'content': 'What is a Python list?'}, {'role': 'assistant', 'content': 'A Python list is a **collection of items** that is **ordered** and **changeable**. Think of it like a shopping list where you can add or remove items, and the order in which you write them down matters.\n\nHere\'s a breakdown of those key characteristics:\n\n*   **Collection of items:** A list can hold multiple values. These values can be of different data types (numbers, strings, even other lists!).\n*   **Ordered:** The items in a list have a specific order. This means you can access items based on their position.\n*   **Changeable (Mutable):** You can modify a list after it\'s created. You can add new items, remove existing ones, or change the value of an item at a particular position.\n\n**How to create a Python list:**\n\nYou create a list by en

In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os
# Load environment variables
load_dotenv()
# Configure client for Gemini API
client = wrap_openai(OpenAI(
 api_key=api_key,
 base_url=api_base,
))

 # Define the tool for weather retrieval
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Chicago, IL",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
            "required": ["location"],
            },
        }
    }
 ]

 # call the chat completion with tool usage
messages = [{"role": "user", "content": "What's the weather like in Chicago today?"}]
response = client.chat.completions.create(
    model="gemini-2.5-flash-lite",
    messages=messages,
    tools=tools
)

 # examine response to see the tool and parameters used.
 # Call response.to_json() for easy to read output
print(response)

ChatCompletion(id='dW_vaLTYLJigxN8Pz8WI2QM', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='function-call-381331813658901517', function=Function(arguments='{"location":"Chicago, IL"}', name='get_weather'), type='function')]))], created=1760522101, model='gemini-2.5-flash-lite', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=84, total_tokens=101, completion_tokens_details=None, prompt_tokens_details=None))


In [9]:
from pydantic import BaseModel, Field
from openai import pydantic_function_tool

class Add(BaseModel):
    """Test tool for addition."""
    a: int = Field(description="First number to add")
    b: int = Field(description="Second number to add")

    def exec(self):
        """Simple addition function."""
        return self.a + self.b


tool = pydantic_function_tool(Add)

print(tool)

{'type': 'function', 'function': {'name': 'Add', 'strict': True, 'parameters': {'description': 'Test tool for addition.', 'properties': {'a': {'description': 'First number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second number to add', 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'Add', 'type': 'object', 'additionalProperties': False}, 'description': 'Test tool for addition.'}}


In [10]:
tool_lookup = {"add": Add}
args = {"a": 5, "b": 7}
res = tool_lookup['add'](**args).exec()
print(res) 

12


In [11]:
from pydantic import BaseModel, Field
class City(BaseModel):
    name: str = Field(description="The name of the city")
    country: str = Field(description="The country where the city is located")
    population: int = Field(description="The population of the city")

response = client.chat.completions.parse(
    model=model,
    messages=[

        {
            "role": "user",
            "content": "Provide details about Paris."
        }
    ],
    response_format=City
)

city = response.choices[0].message.parsed
print(type(city))
print(city)

<class '__main__.City'>
name='Paris' country='France' population=2141000


In [12]:
from pydantic import BaseModel
from openai import OpenAI
client = wrap_openai(OpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta",
    ))
# Define the data model for the math reasoning steps
class Step(BaseModel):
    explanation: str
    output: str

# Define the data model for the overall math reasoning
class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

# Create a chat completion request with a math problem
completion = client.chat.completions.parse(
    model="gemini-2.5-flash-lite",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

 # Parse the response to extract the reasoning steps
solution = completion.choices[0].message.parsed

 # Print the final answer
for step_number, step in enumerate(solution.steps, start=1):
    print(f"Step {step_number}: {step.explanation}")
    print(f"Output: {step.output}\n")


Step 1: The first step is to isolate the term with 'x' by subtracting 7 from both sides of the equation.
Output: 8x + 7 - 7 = -23 - 7

Step 2: Simplify both sides of the equation.
Output: 8x = -30

Step 3: Now, to solve for 'x', divide both sides of the equation by 8.
Output: 8x / 8 = -30 / 8

Step 4: Simplify the fraction.
Output: x = -15 / 4

